<a href="https://colab.research.google.com/github/ericlewisX/trafficsignclassifier/blob/main/Traffic_Signs_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!curl https://sdk.cloud.google.com | bash

In [ ]:
!gcloud init

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list

In [ ]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -

In [ ]:
!apt -qq update

In [ ]:
!apt -qq install gcsfuse

In [ ]:
!mkdir imagedataset

## TensordBoard and Callbacks

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

In [ ]:
early_stopping = EarlyStopping(monitor='loss', 
                               patience=5, mode='auto'
                               )
model_checkpoint = ModelCheckpoint(filepath='./drive/MyDrive/Capstone3/TrafficSigns/modelX.h5', 
                                   monitor='val_accuracy', save_best_only=True)
tensor_board = TensorBoard(log_dir="./logs" )

# Imports

In [ ]:
## Standard
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import pickle
import random

## Image Related
from PIL import Image
from skimage.io import imsave, imshow

## Sklearn
from sklearn.model_selection import train_test_split

## Tensoflow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, models
from tensorflow.keras.layers import Conv2D, UpSampling2D, InputLayer, Dense, \
                                    Conv2DTranspose, MaxPooling2D, Dropout, Flatten, Input
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

path = "imagedataset/TrafficSignClassifier/data/myData/images/"
label_csv = 'imagedataset/TrafficSignClassifier/labels.csv'

In [ ]:
batch_size_val = 50  # how many to process together
steps_per_epoch_val = 2000
epochs_val = 3
imageDimesions = (32,32,3)

test_size = 0.2    # if 1000 images split will 200 for testing
validation_size = 0.2 # if 1000 images 20% of remaining 800 will be 160 for validation


In [5]:
def load_data(path):
  '''
  This function loads all the images from all the directories into lists.
  '''
  class_directory_num, image_list, class_, shapes = 0, [], [], []
  list_ = os.listdir(path) # Number of directories in path is synomous with number of classes
  class_count = len(list_)

  for folder in range(len(list_)):

      sign_list = os.listdir(path + '/' + str(class_directory_num))

      for x in sign_list:

        image = cv2.imread(path + '/' + str(class_directory_num) + '/' + x)
        image_list.append(image)
        class_.append(class_directory_num)
        shapes.append(image.shape)

      # print(class_directory_num, end = ' ')
      class_directory_num += 1

  images_dataset_list = np.array(image_list)
  class_num = np.array(class_)

  return images_dataset_list, class_num, shapes


In [3]:
def save_variables(var_, file_name : str):
  '''
  This function saves variables as binary pickle files for reuse later. This circumvents the
  need to rerun load_data(path) if runtime fails.
  '''
  with open(file_name, 'wb') as f:
    pickle.dump(var_, f)

In [4]:
def load_variables(file_name : str):
  '''
  This function loads variables from a binary pickle file.
  '''
  with open(file_name, 'rb') as f:
    var_ = pickle.load(f)

  return var_
    

In [8]:
def train_test_split(images_dataset_list, class_num, test_size):
  '''
  This function splits our data with a train test split.  
  '''
  X_train, X_test, y_train, y_test = train_test_split(iamges_dataset_list, class_num, test_size = test_size)
  
  return X_train, X_test, y_train, y_test
 

In [9]:
def train_validation_split(X_train, y_train, validation_size):

  '''
  [This function splits our data with a train validation split. ]


  
  Returns
  -------
  [type] - X_train
      [description]
  [type] - X_validation
      [description]
  [type] - y_train
      [description]
  [type] - y_validation
      [description]
  '''

  X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size = validation_size)
 
  return X_train, X_validation, y_train, y_validation 
 
 